Building “dashboard” **inside** a Jupyter notebook (and a few to turn that notebook into a shareable app). Here’s a concise menu of options with when-to-use, pros/cons, and tiny starter snippets.

# Quick chooser

* **Fast in-notebook prototypes:** `ipywidgets` + your plotting lib (Matplotlib/Plotly/Altair), or Plotly `FigureWidget`.
* **Richer, reactive layouts in-notebook:** HoloViz **Panel** (with hvPlot/HoloViews/Bokeh).
* **Dash-style callbacks in-notebook:** **JupyterDash** (Dash inside Jupyter).
* **Drag-and-drop blocks in-notebook:** **Gradio**.
* **Turn the notebook into a clean app:** **Voilà** (best path from notebook → app).
* **Big-data friendly, linked plots:** hvPlot + Datashader + Panel.
* **Power tables:** ipyAGGrid / Perspective widget.

---

# 1) ipywidgets (+ Matplotlib/Plotly/Altair)

**What:** Native Jupyter widgets for controls (dropdowns, sliders, date pickers) with layout containers (VBox/HBox/Tabs/Accordion/GridSpec).
**When:** You want a lightweight dashboard quickly, no new framework.
**Pros:** Zero extra server; works inline.
**Cons:** Layout takes a bit of manual work; not the prettiest out of the box.

In [1]:
import pandas as pd
import ipywidgets as w
from IPython.display import display
import plotly.express as px

df = px.data.gapminder()

country = w.Dropdown(options=sorted(df['country'].unique()), description='Country:')
metric = w.Dropdown(options=['lifeExp','gdpPercap','pop'], description='Metric:')

out = w.Output()

def update(*_):
    with out:
        out.clear_output()
        d = df[df['country']==country.value]
        fig = px.line(d, x='year', y=metric.value, title=f"{metric.value} – {country.value}")
        fig.show()

country.observe(update, 'value')
metric.observe(update, 'value')
update()

ui = w.VBox([w.HBox([country, metric]), out])
display(ui)

**Extras to know:**

* Fancy layouts: `w.GridspecLayout`, `w.Tab`, `w.Accordion`.
* Interactive Matplotlib: enable `%matplotlib widget` (via `ipympl`) for pan/zoom in place.
* Tables: **ipyaggrid** or **perspective** for filter/sort/aggregate.

---

# 2) Plotly FigureWidget (all-Python, reactive traces)

**What:** Plotly object that updates in place—great for brushing & callbacks.
**When:** You love Plotly’s interactivity; want reactive updates without redrawing.
**Pros:** Smooth interactivity, selection events, hover events.
**Cons:** Tight coupling to Plotly.

In [5]:
import plotly.graph_objects as go
from ipywidgets import FloatSlider, VBox

x = list(range(100))
k = FloatSlider(value=1, min=0.1, max=5, step=0.1, description='Scale')

fig = go.FigureWidget()
sc = fig.add_scatter(x=x, y=[i*k.value for i in x])
def on_change(change):
    sc.y = [i*change['new'] for i in x]
k.observe(on_change, names='value')
VBox([k, fig])

    'data': [{'ty…

---
# 3) HoloViz Panel (works beautifully inside notebooks)

**What:** High-level reactive dashboarding with clean layout, binds, and many backends (Bokeh, Matplotlib, Plotly, Altair, HoloViews).
**When:** You want tidy layouts, reactive expressions, and an “upgrade path” to serve as a web app.
**Pros:** Very little glue code; powerful with hvPlot/HoloViews; serve with `panel serve`.
**Cons:** A new mini-ecosystem to learn.

In [5]:
import panel as pn, pandas as pd, hvplot.pandas  # pip install panel hvplot
pn.extension()

df = pd.read_csv('TEST_DATA.csv')

country = pn.widgets.Select(name='Country', options=sorted(df['Country'].unique()))
market  = pn.widgets.Select(name='Market',  options=['Europe','Africa','USCA'])

@pn.depends(country,market)
def view(c, m):
    d = df[df.country==c].sort_values('Order Date')
    return d.hvplot.line(x='Order Date', y=m, title=f"{m} – {c}")

pn.Column(pn.Row(country, market), view)
#> When ready to share the **same notebook** as a web app: `panel serve your_notebook.ipynb`.

AttributeError: 'DataFrame' object has no attribute 'country'

# 4) Bokeh (inline output + widgets)

**What:** Interactive plots + Bokeh widgets; integrates with Panel/HoloViews.
**When:** You like Bokeh’s styling & Python-first callbacks.
**Pros:** Good for linked brushing and custom interactions.
**Cons:** More verbose than Panel/HoloViews.


In [14]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure, ColumnDataSource
from bokeh.models import Slider
from bokeh.layouts import column
output_notebook()

import numpy as np
x = np.linspace(0, 10, 400)
k = Slider(start=1, end=5, step=0.1, value=1, title='Scale')
src = ColumnDataSource(dict(x=x, y=x*k.value))
p = figure(height=300)
p.line('x','y', source=src)

def update(attr, old, new):
    src.data['y'] = x*k.value
k.on_change('value', update)

show(column(k, p))

Loading BokehJS ...

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/js_callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html



---

# 5) JupyterDash (Dash… inside Jupyter)

**What:** Plotly Dash API with callbacks, but runs inline using **JupyterDash**.
**When:** You want Dash-style callbacks and plan to graduate to a real Dash app.
**Pros:** Same code can be served with `app.run_server`.
**Cons:** Callback pattern is more “app-like” than “notebook-like.”


In [9]:
from jupyter_dash import JupyterDash  # pip install jupyter-dash
from dash import html, dcc, Input, Output
import plotly.express as px

df = px.data.gapminder()
app = JupyterDash(__name__)
app.layout = html.Div([
    dcc.Dropdown(sorted(df.country.unique()), df.country.iloc[0], id='country'),
    dcc.Dropdown(['lifeExp','gdpPercap','pop'], 'lifeExp', id='metric'),
    dcc.Graph(id='graph')
])

@app.callback(Output('graph','figure'), [Input('country','value'), Input('metric','value')])
def update(country, metric):
    d = df[df.country==country]
    return px.line(d, x='year', y=metric, title=f'{metric} – {country}')

app.run_server(mode='inline', debug=False)

C:\Users\Lenovo\anaconda3\Lib\site-packages\dash\dash.py:587: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



AttributeError: 'super' object has no attribute 'run_server'

# 6) Gradio (blocks UI in a cell)

**What:** Simple blocks (inputs/outputs) that render a mini web app right in the notebook.
**When:** You want dead-simple UIs (selectors, sliders, upload) for functions/ML demos.
**Pros:** Fastest to demo; one function → UI.
**Cons:** Less dashboard-y layouts, more “tool UI”.

In [3]:
import gradio as gr
import pandas as pd

def filter_top_n(n):
    df = pd.DataFrame({'A':[1,2,3,4,5],'B':[5,4,3,2,1]})
    return df.head(n)

demo = gr.Interface(fn=filter_top_n, inputs=gr.Slider(1,5,step=1), outputs='dataframe')
demo.launch(inline=True)

ModuleNotFoundError: No module named 'gradio'

---

# 7) Turn the notebook into a standalone app (no code cells)

**Voilà**

* **What:** Renders your notebook as a web app—shows outputs/widgets, hides code cells.
* **How:** Add `ipywidgets`/Panel content in the notebook → run:

  ```
  voila your_notebook.ipynb          # local
  ```
* **Deploy:** Binder, JupyterHub, or a small server (`pip install voila`).
* **Pros:** You keep working in notebooks; users see a clean app.
* **Cons:** Best with widget-based UIs; advanced auth/routing needs extra setup.

**Alternatives:**

* **Panel serve**: `panel serve notebook.ipynb` (turns Panel content into a served app).
* **Mercury** (MLJAR): turn notebooks into parameterized web apps.
* **jupyter-appmode**: hides code cells, app-like feel (lighter option).

---

# 8) Big data + linked views

If you need **millions of points** and fluid interaction:

* **hvPlot + Datashader + Panel** for auto-aggregated, GPU-friendly plots.
* **Perspective** (Finos) widget for pivoting, grouping, and lightning-fast tables.

---

# 9) Geospatial dashboards

* **ipyleaflet** + ipywidgets for map layers, markers, draw tools.
* **Kepler.gl for Jupyter** or **Folium** (more static) for quick choropleths.

---

# 10) Layout & table building blocks

* **ipywidgets containers:** `VBox`, `HBox`, `GridBox`, `GridspecLayout`, `Tab`, `Accordion`.
* **Tables:** `ipyaggrid`, `qgrid` (older), **perspective** (very fast), or **plotly.data\_table**.

---

# 11) Packaging & sharing tips

* Put notebook + a small `env.yml` or `requirements.txt` so others can reproduce.
* For Voilà/Panel/Dash apps, keep data loading in top cells and **cache** heavy steps (e.g., with `joblib.Memory` or `functools.lru_cache`).
* Separate “data prep” cells from “UI” cells; consider a tiny `utils.py` for reusable code.
* If you’ll later move beyond Jupyter, the closest upgrade paths are:

  * **Panel → panel serve** (no rewrite),
  * **JupyterDash → Dash server**,
  * **Gradio → gradio app**.

---

## If you want a starting template (recommended)

Tell me your preferred stack (e.g., **ipywidgets**, **Panel**, or **JupyterDash**) and the kind of charts/tables you need. I’ll drop a minimal, well-commented notebook scaffold you can paste in and run.